## Load PSQL geodb service

In [1]:
from xcube_sh.geodb import get_geo_db_service
geo = get_geo_db_service(driver='pgsql', host="db-dcfs-geodb.cbfjgqxk302m.eu-central-1.rds.amazonaws.com",
            user="postgres",
            password="Oeckel6b&z")

## Find features

Find all features

In [2]:
fs = geo.find_feature(collection_name='germany_sh_lakes', query="'S_NAME' = 'Hemmelmarker_See'")

In [3]:
from IPython.display import GeoJSON

In [4]:
GeoJSON(fs)

<IPython.display.GeoJSON object>

## Use local geodb

In [5]:
geo_local = get_geo_db_service(driver='local')

In [6]:
fs = geo_local.find_feature(collection_name='germany-sh-lakes', query="S_NAME == 'Hemmelmarker_See'")

In [7]:
GeoJSON(fs)

<IPython.display.GeoJSON object>

## Create a new collection

In [8]:
fs = geo.find_features(collection_name='germany_sh_lakes', query="'S_NAME' LIKE '%'", max_records=200)

In [11]:
f = fs[0]
schema = {'properties': {}}
for k, v in f.items():
    try:
        float(f[k])
        schema['properties'][k] = 'float'
    except:
        schema['properties'][k] = 'str'

In [42]:
geo.new_collection('germany_sh_lakes_long', schema=schema)

ValueError: Collection germany_sh_lakes_long exists

In [4]:
geo.add_features(collection_name='germany_sh_lakes_long', features=fs)

'Features Added'

In [14]:
geo.find_features(collection_name='germany_sh_lakes_long', query="name LIKE '%'", fmt='gdf')

,id,properties,name,geometry,metadata_u,restricted,template,s_name,eu_cd_ls,eu_cd_lw,...,qe3162monr,qe3162qegr,qe3162val,qe33chang,qe33comp,qe33monp,qe33monr,qe33qegrou,qe33val,numreferen
0,0,"{'id': '0', 'METADATA_U': 'http://www.wasserbl...",Hemmelmarker_See,"POLYGON Z ((9.89765 54.49069 0.00000, 9.89760 ...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Hemmelmarker_See,DE_LS_DESH_0145,DE_LW_DESH_0145,...,None,None,3.0,3.0,None,2008--2013,None,None,2.0,None
1,1,"{'id': '1', 'METADATA_U': 'http://www.wasserbl...",Langsee_Suederfahrenstedt,"POLYGON Z ((9.53135 54.57168 0.00000, 9.53138 ...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Langsee_Suederfahrenstedt,DE_LS_DESH_0228,DE_LW_DESH_0228,...,None,None,3.0,3.0,None,2008--2013,None,None,2.0,None
2,31,"{'id': '31', 'METADATA_U': 'http://www.wasserb...",Westensee,"MULTIPOLYGON Z (((9.91504 54.29099 0.00000, 9....",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Westensee,DE_LS_DESH_0443,DE_LW_DESH_0443,...,None,None,3.0,3.0,None,2008--2013,None,None,2.0,None
3,2,"{'id': '2', 'METADATA_U': 'http://www.wasserbl...",Schaalsee_davon_Lassahner_See,"POLYGON Z ((10.94771 53.59346 0.00000, 10.9474...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Schaalsee_davon_Lassahner_See,DE_LS_DEMV_LS_2300103,DE_LW_DEMV_2300103,...,None,None,6.0,7.0,5,2010--2014,None,None,2.0,None
4,3,"{'id': '3', 'METADATA_U': 'http://www.wasserbl...",Schoehsee,"POLYGON Z ((10.43024 54.16311 0.00000, 10.4302...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Schoehsee,DE_LS_DESH_0359,DE_LW_DESH_0359,...,None,None,2.0,3.0,None,2008--2013,None,None,2.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,80,"{'id': '80', 'METADATA_U': 'http://www.wasserb...",Speicherbecken_Miele,"POLYGON Z ((8.98836 54.09468 0.00000, 8.98907 ...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Speicherbecken_Miele,DE_LS_DESH_0535,DE_LW_DESH_0535,...,None,None,6.0,3.0,None,2008--2013,None,None,2.0,None
89,81,"{'id': '81', 'METADATA_U': 'http://www.wasserb...",Grosser_Binnensee,"POLYGON Z ((10.63522 54.33562 0.00000, 10.6353...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Grosser_Binnensee,DE_LS_DESH_0109,DE_LW_DESH_0109,...,None,None,3.0,3.0,None,2008--2013,None,None,2.0,None
90,82,"{'id': '82', 'METADATA_U': 'http://www.wasserb...",Grosser_Poenitzer_See,"POLYGON Z ((10.69631 54.03813 0.00000, 10.6964...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Grosser_Poenitzer_See,DE_LS_DESH_0115,DE_LW_DESH_0115,...,None,None,2.0,3.0,None,2008--2013,None,None,2.0,None
91,83,"{'id': '83', 'METADATA_U': 'http://www.wasserb...",Neustaedter_Binnenwasser_Ost,"MULTIPOLYGON Z (((10.79591 54.11351 0.00000, 1...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Neustaedter_Binnenwasser_Ost,DE_LS_DESH_0284,DE_LW_DESH_0284,...,None,None,3.0,3.0,None,2008--2013,None,None,2.0,None
